In [2]:
import pandas as pd
import json

/Users/mats_dodd/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [6]:
data = pd.read_json("../../llm/commitees/committee_mappings.json")

# Initialize lists to hold the flattened data
committees = []
sessions = []
subcommittees = []

# Iterate over each committee and session to extract subcommittees
for committee, row in data.iterrows():
    for session, subcommittee_list in row.items():
        for subcommittee in subcommittee_list:
            committees.append(committee)
            sessions.append(session)
            subcommittees.append(subcommittee)

# Convert lists to a pandas DataFrame
flattened_df = pd.DataFrame({
    'Committee': committees,
    'Session': sessions,
    'Subcommittee': subcommittees
})

flattened_df.head()


,Committee,Session,Subcommittee
0,"AGRICULTURE, NUTRITION, AND FORESTRY",109,"FORESTRY, CONSERVATION, AND RURAL REVITALIZATION"
1,"AGRICULTURE, NUTRITION, AND FORESTRY",109,"MARKETING, INSPECTION, AND PRODUCT PROMOTION"
2,"AGRICULTURE, NUTRITION, AND FORESTRY",109,PRODUCTION AND PRICE COMPETITIVENESS
3,"AGRICULTURE, NUTRITION, AND FORESTRY",109,"RESEARCH, NUTRITION, AND GENERAL LEGISLATION"
4,"AGRICULTURE, NUTRITION, AND FORESTRY",110,"DOMESTIC AND FOREIGN MARKETING, INSPECTION, AN..."


In [3]:


with open('../../llm/commitees/committee_mappings.json', 'r') as file:
    data = json.load(file)

In [4]:
import ast

def destring_dict_values(d):
    return {k: ast.literal_eval(v) if isinstance(v, str) else v for k, v in d.items()}



In [5]:

converted_dict = destring_dict_values(data)

converted_dict

{'109': {'AGRICULTURE, NUTRITION, AND FORESTRY': ['FORESTRY, CONSERVATION, AND RURAL REVITALIZATION',
   'MARKETING, INSPECTION, AND PRODUCT PROMOTION',
   'PRODUCTION AND PRICE COMPETITIVENESS',
   'RESEARCH, NUTRITION, AND GENERAL LEGISLATION'],
  'APPROPRIATIONS': ['AGRICULTURE, RURAL DEVELOPMENT, AND RELATED AGENCIES',
   'COMMERCE, JUSTICE, SCIENCE AND RELATED AGENCIES',
   'DEFENSE',
   'DISTRICT OF COLUMBIA',
   'ENERGY AND WATER, AND RELATED AGENCIES',
   'HOMELAND SECURITY',
   'INTERIOR AND RELATED AGENCIES',
   'LABOR, HEALTH AND HUMAN SERVICES, EDUCATION, AND RELATED AGENCIES',
   'LEGISLATIVE BRANCH',
   'MILITARY CONSTRUCTION AND VETERANS’ AFFAIRS AND RELATED AGENCIES',
   'STATE, FOREIGN OPERATIONS, AND RELATED PROGRAMS',
   'TRANSPORTATION, TREASURY, THE JUDICIARY, AND HOUSING AND URBAN DEVELOPMENT, AND RELATED AGENCIES'],
  'ARMED SERVICES': ['AIRLAND',
   'EMERGING THREATS AND CAPABILITIES',
   'PERSONNEL',
   'READINESS AND MANAGEMENT SUPPORT',
   'SEAPOWER',
   'STR

In [33]:

dfs = []

for senator in converted_dict['118']:
    # Create a DataFrame for each senator
    df = pd.DataFrame([senator])

    # Explode committees and subcommittees columns
    df = df.explode('committees').explode('subcomittees').reset_index(drop=True)

    # Pivot the DataFrame to have committees and subcommittees as columns
    df_committees = df.pivot(columns='committees', values='committees').fillna(0)
    df_committees = df_committees.applymap(lambda x: 1 if x != 0 else 0)

    df_subcomittees = df.pivot(columns='subcomittees', values='subcomittees').fillna(0)
    df_subcomittees = df_subcomittees.applymap(lambda x: 1 if x != 0 else 0)

    # Merge the pivoted DataFrames
    df = pd.concat([df[['full_name', 'state']], df_committees, df_subcomittees], axis=1).groupby(['full_name', 'state']).sum().reset_index()

    dfs.append(df)

# Combine all the senator DataFrames
result_df_118 = pd.concat(dfs, axis=0).reset_index(drop=True)
result_df_118

,full_name,state,"AGRICULTURE, NUTRITION, AND FORESTRY",BUDGET,ENVIRONMENT AND PUBLIC WORKS,FINANCE,"CLEAN AIR, CLIMATE, AND NUCLEAR SAFETY","ENERGY, NATURAL RESOURCES, AND INFRASTRUCTURE","FISHERIES, WATER, AND WILDLIFE",HEALTH CARE,...,NATIONAL SECURITY AND INTERNATIONAL TRADE AND FINANCE,EUROPE AND REGIONAL SECURITY COOPERATION,"MULTILATERAL INTERNATIONAL DEVELOPMENT, MULTILATERAL INSTITUTIONS, AND INTERNATIONAL ECONOMIC, ENERGY, AND ENVIRONMENTAL POLICY","WESTERN HEMISPHERE, TRANSNATIONAL CRIME, CIVILIAN SECURITY, DEMOCRACY, HUMAN RIGHTS, AND GLOBAL WOMEN’S ISSUES","EAST ASIA, THE PACIFIC, AND INTERNATIONAL CYBERSECURITY POLICY","PUBLIC LANDS, FORESTS, AND MINING",GOVERNMENT OPERATIONS AND BORDER MANAGEMENT,PERSONNEL,EMERGING THREATS AND SPENDING OVERSIGHT,
0,Debbie Stabenow,Michigan,6.0,6.0,6.0,6.0,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sherrod Brown,Ohio,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amy Klobuchar,Minnesota,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Michael F. Bennet,Colorado,5.0,NaN,NaN,5.0,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kirsten E. Gillibrand,New York,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,John Cornyn,Texas,NaN,NaN,NaN,8.0,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,James Lankford,Oklahoma,NaN,NaN,NaN,5.0,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN
97,Rand Paul,Kentucky,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Jon Ossoff,Georgia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN


In [34]:
dfs = []

for senator in converted_dict['117']:
    # Create a DataFrame for each senator
    df = pd.DataFrame([senator])

    # Explode committees and subcommittees columns
    df = df.explode('committees').explode('subcomittees').reset_index(drop=True)

    # Pivot the DataFrame to have committees and subcommittees as columns
    df_committees = df.pivot(columns='committees', values='committees').fillna(0)
    df_committees = df_committees.applymap(lambda x: 1 if x != 0 else 0)

    df_subcomittees = df.pivot(columns='subcomittees', values='subcomittees').fillna(0)
    df_subcomittees = df_subcomittees.applymap(lambda x: 1 if x != 0 else 0)

    # Merge the pivoted DataFrames
    df = pd.concat([df[['full_name', 'state']], df_committees, df_subcomittees], axis=1).groupby(['full_name', 'state']).sum().reset_index()

    dfs.append(df)

# Combine all the senator DataFrames
result_df_117 = pd.concat(dfs, axis=0).reset_index(drop=True)
result_df_117

,full_name,state,"AGRICULTURE, NUTRITION, AND FORESTRY",BUDGET,ENVIRONMENT AND PUBLIC WORKS,FINANCE,"CLEAN AIR, CLIMATE, AND NUCLEAR SAFETY","FISHERIES, WATER, AND WILDLIFE",HEALTH CARE,"INTERNATIONAL TRADE, CUSTOMS, AND GLOBAL COMPETITIVENESS",...,"WESTERN HEMISPHERE, TRANSNATIONAL CRIME, CIVILIAN SECURITY, DEMOCRACY, HUMAN RIGHTS, AND GLOBAL WOMEN’S ISSUES","EAST ASIA, THE PACIFIC, AND INTERNATIONAL CYBERSECURITY POLICY","STATE DEPARTMENT AND USAID MANAGEMENT, INTERNATIONAL OPERATIONS, AND BILATERAL INTERNATIONAL DEVELOPMENT",AIRLAND,FISCAL RESPONSIBILITY AND ECONOMIC GROWTH,HOMELAND SECURITY AND GOVERNMENTAL AFFAIRS,EMERGING THREATS AND SPENDING OVERSIGHT,PERMANENT SUBCOMMITTEE ON INVESTIGATIONS,GOVERNMENT OPERATIONS AND BORDER MANAGEMENT,
0,Debbie Stabenow,Michigan,6.0,6.0,6.0,6.0,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Patrick J. Leahy,Vermont,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sherrod Brown,Ohio,6.0,NaN,NaN,6.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Amy Klobuchar,Minnesota,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Michael F. Bennet,Colorado,6.0,NaN,NaN,6.0,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,John Cornyn,Texas,NaN,NaN,NaN,7.0,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Richard Burr,North Carolina,NaN,NaN,NaN,3.0,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
97,Rob Portman,Ohio,NaN,NaN,NaN,6.0,NaN,NaN,NaN,3.0,...,3.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN
98,Rand Paul,Kentucky,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,8.0,4.0,4.0,NaN,NaN
